# 견적서 HQ

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)
import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob
import json
from tqdm.notebook import tqdm

## 1) 견적서 json 가져오기

In [15]:
label_list = glob.glob("C:\\eun01\\car_repair\\training\\as_labeling_data\\*")
print(len(label_list))

125006


In [16]:
label_list[40].split("\\")[-1][:-5]

'as-0000076'

In [4]:
with open(label_list[0],'rt', encoding='UTF8') as file:
    contents=json.loads(file.read())
contents["수리내역"][0]["작업항목 및 부품명"]

'앞범퍼(스틸형)하단'

In [5]:
contents["수리내역"][0]

{'No': '1 U',
 '작업항목 및 부품명': '앞범퍼(스틸형)하단',
 '작업': '교환',
 'HQ%': 1.24,
 '부품코드': '',
 '부품가격': '',
 '공임': '40,920',
 '비고': ''}

In [17]:
category_id=[]
part=[]
repair=[]
HQ=[]

for file_name in tqdm(label_list):
    with open(file_name,'rt', encoding='UTF8') as file:
        contents=json.loads(file.read())
    for i in range(len(contents["수리내역"])):
        category_id.append(file_name.split("\\")[-1][:-5])
        part.append(contents["수리내역"][i]["작업항목 및 부품명"])
        repair.append(contents["수리내역"][i]["작업"])
        HQ.append(contents["수리내역"][i]["HQ%"])

data = pd.DataFrame({"category_id" : category_id, "part" : part, "repair" : repair, "HQ" : HQ})

  0%|          | 0/125006 [00:00<?, ?it/s]

In [18]:
data.head()

,category_id,part,repair,HQ
0,as-0000002,앞범퍼(스틸형)하단,교환,1.24
1,as-0000002,앞범퍼,교환,3.01
2,as-0000002,헤드램프(좌),탈착,0.42
3,as-0000002,헤드램프(우),탈착,0.42
4,as-0000002,앞패널,판금,2.5


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716713 entries, 0 to 1716712
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   category_id  object
 1   part         object
 2   repair       object
 3   HQ           object
dtypes: object(4)
memory usage: 52.4+ MB


#### 외제차 제외

In [24]:
kor_index = pd.read_csv("kor_car.csv")
kor_index.rename(columns = {"file_name" : 'category_id'}, inplace = True)

In [25]:
data = pd.merge(data, kor_index, how='inner', on='category_id')

## 2) 작업 방법 전처리

    도장 coating -> 도장공임
    판금 sheet_metal -> 판금수리
    교환 exchange -> 탈착교환
    수리 repair -> ?

In [26]:
data["repair"].unique()

array(['교환', '탈착', '판금', '1/2OH', '도장', '수리', '', '오버홀', '조정', '견인',
       '1/4OH', '1/3OH', '견인비', '1/2오버홀', '1/4오버홀', '구난비', '구난', '1/3오버홀',
       '불인정', '작업'], dtype=object)

In [11]:
print(len(data[data["repair"]=="교환"])) # exchange -> 탈착교환
print(len(data[data["repair"]=="탈착"]))

print(len(data[data["repair"]=="도장"])) # coating -> 도장공임

print(len(data[data["repair"]=="판금"])) # 판금 sheet_metal -> 판금수리
print(len(data[data["repair"]=="수리"]))

309889
456744
460961
123644
56610


In [12]:
print(len(data[data["repair"]=="오버홀"]))
print(len(data[data["repair"]=="1/2OH"]))
print(len(data[data["repair"]=="1/3OH"]))
print(len(data[data["repair"]=="1/4OH"]))
print(len(data[data["repair"]=="1/2오버홀"]))
print(len(data[data["repair"]=="1/3오버홀"]))
print(len(data[data["repair"]=="1/4오버홀"]))

21753
22723
3334
10174
10
8
37


In [27]:
#### 해당 작업 항목 행 삭제 ####
data_drop = data.copy()
drop_list = ["", "구난", "구난비", "불인정", "차감소계", "감가상각", "작업", "견인", "견인비", 
             "1/2OH", "오버홀", "조정", "1/4OH", "1/3OH", "1/2오버홀", "1/4오버홀", "1/3오버홀"]

for drop in drop_list:
    data_drop = data_drop.drop(data_drop[data_drop["repair"]==drop].index, axis=0)

In [28]:
data_drop = data_drop.reset_index()
data_drop.drop("index", axis=1, inplace=True)

In [29]:
data_drop["repair"].unique()

array(['교환', '탈착', '판금', '도장', '수리'], dtype=object)

In [32]:
print(len(data))
print(len(data_drop))

1675156
1407848


In [21]:
data_drop["part"].value_counts()

가열건조비                    112622
공통시간작업                    78213
후론트 범퍼 커버                 55689
리어 컴비네이션램프(우)             29964
리어 컴비네이션램프(좌)             28266
                          ...  
좌)사이드스텝발판                     1
리어범퍼하단커버교환                    1
우)전도어,뒤도어,뒤휀더-폴리싱             1
사이드스텝패널(아우터,우)-폴리싱            1
프런트도어벨트몰딩(눈썹몰딩)(좌)재사용         1
Name: part, Length: 20437, dtype: int64

In [20]:
print(len(list(data["part"].unique())))
print(len(list(data_drop["part"].unique())))

46493
20437


In [33]:
part = pd.DataFrame({"part":list(data_drop["part"].unique())})
part.head()

,part
0,후론트 범퍼
1,헤드램프(우)
2,후론트 휀다(우)
3,후론트 도어(우)
4,슬라이딩 도어(우)


In [ ]:
# 작업을 위한 분류 #
# "Bumper", "Bonnet", "Roof", "Head lights", "Door", "Fender", "Rocker panel", "Side mirror", "Trunk lid", "Rear lamp", "Wheel" #
part1 = part[:4000]
part2 = part[4000:8000]
part3 = part[8000:12000]
part4 = part[12000:16000]
part5 = part[16000:]

part1.to_csv("part1.csv", index=False, encoding="euc-kr")
part2.to_csv("part2.csv", index=False, encoding="euc-kr")
part3.to_csv("part3.csv", index=False, encoding="euc-kr")
part4.to_csv("part4.csv", index=False, encoding="euc-kr")
part5.to_csv("part5.csv", index=False, encoding="euc-kr")

## 3) 작업항목 및 부품 전처리

In [34]:
part1 = pd.read_csv("part1_part.csv", encoding="euc-kr")
part2 = pd.read_csv("part2_part.csv", encoding="euc-kr")
part3 = pd.read_csv("part3_part.csv", encoding="euc-kr")
part4 = pd.read_csv("part4_part.csv", encoding="euc-kr")
part5 = pd.read_csv("part5_part.csv", encoding="euc-kr")

In [35]:
part2 = part2[["part", "Unnamed: 1"]]
part3.rename(columns = {"part.1":"Unnamed: 1"}, inplace = True)

In [36]:
print(len(part1))
print(len(part2))
print(len(part3))
print(len(part4))
print(len(part5))

1042
940
517
1131
1631


In [37]:
part_cc = pd.concat([part1, part2, part3, part4, part5], axis=0, ignore_index=True)
part_cc.head()

,part,Unnamed: 1
0,후론트 범퍼,Bumper
1,헤드램프(우),Head lights
2,후론트 휀다(우),Fender
3,후론트 도어(우),Door
4,슬라이딩 도어(우),Door


In [38]:
HQ_data = pd.merge(data_drop, part_cc, how='inner', on="part")
HQ_data.head()

,category_id,part,repair,HQ,Unnamed: 1
0,as-0000025,후론트 범퍼,교환,1.79,Bumper
1,as-0000596,후론트 범퍼,교환,2.45,Bumper
2,as-0001501,후론트 범퍼,탈착,1.5,Bumper
3,as-0001551,후론트 범퍼,탈착,0.81,Bumper
4,as-0001576,후론트 범퍼,탈착,0.81,Bumper


In [39]:
HQ_data = HQ_data[["category_id", "Unnamed: 1", "repair", "HQ"]]
HQ_data.rename(columns = {"Unnamed: 1" : "part"}, inplace = True)
HQ_data.head()

,category_id,part,repair,HQ
0,as-0000025,Bumper,교환,1.79
1,as-0000596,Bumper,교환,2.45
2,as-0001501,Bumper,탈착,1.5
3,as-0001551,Bumper,탈착,0.81
4,as-0001576,Bumper,탈착,0.81


In [41]:
# 작업 방식 이름 변경 #
idx = HQ_data[HQ_data["repair"]=="교환"].index
HQ_data["repair"][idx] = "exchange"
idx = HQ_data[HQ_data["repair"]=="탈착"].index
HQ_data["repair"][idx] = "exchange"

idx = HQ_data[HQ_data["repair"]=="도장"].index
HQ_data["repair"][idx] = "coating"

idx = HQ_data[HQ_data["repair"]=="판금"].index
HQ_data["repair"][idx] = "sheet_metal"
idx = HQ_data[HQ_data["repair"]=="수리"].index
HQ_data["repair"][idx] = "sheet_metal"

In [43]:
HQ_data["repair"].unique()

array(['exchange', 'sheet_metal', 'coating'], dtype=object)

In [44]:
# 부품명 통일 #
idx = HQ_data[HQ_data["part"]=="door"].index
HQ_data["part"][idx] = "Door"

In [45]:
HQ_data["part"].unique()

array(['Bumper', 'Head lights', 'Fender', 'Door', 'Wheel', 'Rocker panel',
       'Rear lamp', 'Side mirror', 'Trunk lid', 'Bonnet', 'Roof'],
      dtype=object)

## 4) 학습시 필요한 항목 추가

In [47]:
car_repair1 = pd.read_csv("car_repair1.csv")
car_repair2 = pd.read_csv("car_repair2.csv")
car_repair3 = pd.read_csv("car_repair3.csv")

In [48]:
all_data = pd.concat([car_repair1, car_repair2, car_repair3], ignore_index=True)
all_data.sort_values("category_id")
all_data.head()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
0,2,as-0036229,"[[[[438, 504], [440, 443], [436, 419], [439, 4...",14977.0,"[336, 159, 198, 345]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
1,3,as-0036229,"[[[[469, 497], [448, 479], [458, 437], [461, 4...",7186.5,"[378, 209, 154, 288]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
2,4,as-0036229,"[[[[73, 596], [69, 588], [67, 582], [65, 575],...",749.5,"[53, 543, 31, 53]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
3,4,as-0036229,"[[[[107, 371], [108, 362], [445, 361], [795, 3...",13872.5,"[107, 356, 688, 29]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
4,5,as-0036229,"[[[[77, 390], [87, 375], [452, 351], [560, 356...",22816.0,"[77, 351, 690, 52]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car


In [49]:
# 외제차 분류 #
merge_car = pd.merge(all_data[["category_id", "damage", "supercategory_name"]], kor_index, how='inner',on='category_id')
merge_car.head()

,category_id,damage,supercategory_name
0,as-0036229,Breakage,Full-size car
1,as-0036229,Breakage,Full-size car
2,as-0036229,Breakage,Full-size car
3,as-0036229,Scratched,Full-size car
4,as-0036229,Scratched,Full-size car


In [50]:
# null값 처리 #
null_list = merge_car[merge_car["damage"].isnull()].index
merge_car = merge_car.drop(null_list, axis=0)
merge_car.sort_values("category_id", inplace=True, ignore_index=True)
merge_car.head()

,category_id,damage,supercategory_name
0,as-0000025,Crushed,Full-size car
1,as-0000025,Separated,Full-size car
2,as-0000025,Scratched,Full-size car
3,as-0000025,Crushed,Full-size car
4,as-0000025,Scratched,Full-size car


In [51]:
merge_car["damage"].unique()

array(['Crushed', 'Separated', 'Scratched', 'Breakage'], dtype=object)

In [52]:
# 경형을 소형으로 합체 #
idx = merge_car[merge_car["supercategory_name"]=="CityCar"].index
merge_car["supercategory_name"][idx] = "Compact car"

In [53]:
# HQ와 사고유형 합체 #
merge_data = pd.merge(HQ_data, merge_car, how='inner',on='category_id')
merge_data.head()

,category_id,part,repair,HQ,damage,supercategory_name
0,as-0000025,Bumper,exchange,1.79,Crushed,Full-size car
1,as-0000025,Bumper,exchange,1.79,Separated,Full-size car
2,as-0000025,Bumper,exchange,1.79,Scratched,Full-size car
3,as-0000025,Bumper,exchange,1.79,Crushed,Full-size car
4,as-0000025,Bumper,exchange,1.79,Scratched,Full-size car


In [54]:
merge_data = merge_data[["category_id", "part", "damage", "supercategory_name", "repair", "HQ"]]

In [55]:
# 중복 없애주기 #
m_data = merge_data.drop_duplicates(keep = 'first', ignore_index=True)
m_data.head()

,category_id,part,damage,supercategory_name,repair,HQ
0,as-0000025,Bumper,Crushed,Full-size car,exchange,1.79
1,as-0000025,Bumper,Separated,Full-size car,exchange,1.79
2,as-0000025,Bumper,Scratched,Full-size car,exchange,1.79
3,as-0000025,Head lights,Crushed,Full-size car,exchange,0.44
4,as-0000025,Head lights,Separated,Full-size car,exchange,0.44


In [57]:
print(len(HQ_data))
print(len(m_data))

647233
1257229


In [58]:
# 작업 항목별로 분류 #
exchange = m_data[m_data["repair"]=="exchange"]
exchange.sort_values("category_id", inplace=True, ignore_index=True)

coating = m_data[m_data["repair"]=="coating"]
coating.sort_values("category_id", inplace=True, ignore_index=True)

sheet_metal = m_data[m_data["repair"]=="sheet_metal"]
sheet_metal.sort_values("category_id", inplace=True, ignore_index=True)

print(len(exchange))
print(len(coating))
print(len(sheet_metal))

C:\Users\acorn\AppData\Local\Temp\ipykernel_3240\4233201871.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange.sort_values("category_id", inplace=True, ignore_index=True)
C:\Users\acorn\AppData\Local\Temp\ipykernel_3240\4233201871.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coating.sort_values("category_id", inplace=True, ignore_index=True)


545267
474761
237201


C:\Users\acorn\AppData\Local\Temp\ipykernel_3240\4233201871.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet_metal.sort_values("category_id", inplace=True, ignore_index=True)


In [66]:
exchange.to_csv("HQ_exchange.csv", index=False)
coating.to_csv("HQ_coating.csv", index=False)
sheet_metal.to_csv("HQ_sheet_metal.csv", index=False)